# Disease prediction System using chatbot

In [12]:
import pickle
import nltk
import pandas as pd
import numpy as np
import speech_recognition as sr
from sklearn.pipeline import Pipeline
from rake_nltk import Rake
import pyttsx3
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

In [13]:
intro = pd.read_csv('response//intro_db.csv')
prec = pd.read_csv('response//prec_db.csv')
desc = pd.read_csv('response//desc_db.csv')
symp_greet = pd.read_csv('response//symp_greet_db.csv')
symp = pd.read_csv('response//symp_db.csv')
symp_error = pd.read_csv('response//symp_error_db.csv')
bye = pd.read_csv('response//bye_db.csv')


des = pd.read_csv('csv/symptom_Description.csv')
prec = pd.read_csv('csv/symptom_precaution.csv')
sev = pd.read_csv('csv/Symptom-severity.csv')

In [14]:
with open("CPS.pkl", 'rb') as file:  
    CPS_Model = pickle.load(file)

chatbot = ChatBot("Chatterbot",storage_adapter="chatterbot.storage.SQLStorageAdapter")

trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.english")


Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [15]:
def speech ():  
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Talk")
        audio_input = r.record(source, duration= 6)
        try:
            audio_text = r.recognize_google(audio_input)
            print("User: ",audio_text)
            return (audio_text)
        except:
             sample_n(symp_error)
def t2s (text,sp):
    engine = pyttsx3.init()
    text = np.array(text)
    voices = engine.getProperty('voices') 
    engine.setProperty("rate", sp)
    engine.setProperty("voice", voices[1].id)
    engine.say(text)
    print('Hekka : ',str(text))
    return (engine.runAndWait())                

In [16]:
class RakeImpl:
        def __init__(self, text):
            self.text = text
            self.rake = Rake()
        def getKeywords(self):
            self.rake.extract_keywords_from_text(self.text)
            return self.rake.get_ranked_phrases()

In [17]:
def preprocess(user_text):    
        rakeImpl = RakeImpl(user_text)
        user_text = rakeImpl.getKeywords()[:20]
        user_text = ''.join(user_text)
        return user_text

In [18]:
def chat_resp(ques):
    response = chatbot.get_response(ques)
    return (response)
def random_resp(df):
    text = df['0'].sample(n=1)
    text = np.array(text)
    return (''.join(text))   
def convert(text):
    text = np.array(text)
    return (''.join(text))

In [19]:
def predict (symp):
        symp = ' '+str(''.join(user_symp) )+ symp
        user_symp.clear()
        symp = preprocess(symp)
        user_symp.append(symp)
        print(user_symp)
        y_pred = CPS_Model.predict_proba([symp])
        pred = CPS_Model.predict([symp])
        return pred,y_pred.max()

In [20]:
def diagnose_user():
    resp = random_resp(symp)
    t2s(resp,170)
    text = speech()
    pre, pre_prob = predict(text)
    while pre_prob < 0.85:
        resp = random_resp(symp_error)
        t2s(resp,170)
        text = speech()
        if 'stop' in text:
            user_symp.clear()
            resp = "Diagnosing process was stopped !!"
            t2s(resp,170)
            diagnose = False
            break
        pre, pre_prob = predict(text)
    dis(pre,pre_prob)
    
def dis(pre,pre_prob): 
    pre_prob = pre_prob * 100
    resp = "I predicted you might "+ str(pre) + ", with probability of "+ '%.2f'%pre_prob
    t2s(resp,170)
    desc_cure(pre)

def desc_cure(pre):
    pre = ''.join(pre)
    val = des[des['Disease'].str.lower() == pre]
    print('\nDescription about predicted disease\n')
    text = convert(val.Description)
    t2s(text,190)
    val = prec[prec['Disease'].str.lower() == pre]
    print('\nPrecautions for predicted disease\n')
    text = convert(val.Precaution_1)
    t2s(text,170)
    text =convert(val.Precaution_2)
    t2s(text,170)
    text = convert(val.Precaution_3)
    t2s(text,170)
    text =convert(val.Precaution_4)
    t2s(text,170)


In [21]:
user_symp =[]
print('Say "START" to start diagnosing process')
print('Say "STOP" to stop diagnosing process')
print('Say "QUIT" to exit chatbot\n')
resp = random_resp(intro)
t2s(resp,170)
state = 'online'

while state!='offline':
    text = speech()
    
    if 'start' in text:
        diagnose_user()
        
    
    elif 'quit' in text:
        user_symp=[]
        resp = random_resp(bye)
        t2s(resp,170)
        state = 'offline'
    else:
        user_symp=[]
        resp = chat_resp(text)
        t2s(resp,170) 
    

Say "START" to start diagnosing process
Say "STOP" to stop diagnosing process
Say "QUIT" to exit chatbot

Hekka :  I'm Hekka! your MED-BOT
Talk
User:  hello
Hekka :  Hi there!
Talk
User:  start diagnosing
Hekka :  I can help you find out what's going on with a simple symptom assisment.
Talk
User:  I have running nose fever headache
['running nose fever headache']
Hekka :  Are you suffering from body pain ? I need more informations.
Talk
User:  stop diagnosing
Hekka :  Diagnosing process was stopped !!
Hekka :  I predicted you might ['impetigo'], with probability of 35.54

Description about predicted disease

Hekka :  Impetigo (im-puh-TIE-go) is a common and highly contagious skin infection that mainly affects infants and children. Impetigo usually appears as red sores on the face, especially around a child's nose and mouth, and on hands and feet. The sores burst and develop honey-colored crusts.

Precautions for predicted disease

Hekka :  soak affected area in warm water
Hekka :  use 